In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [20]:
data = pd.read_csv(r'../Data/Sentiment.csv')
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [21]:
data.isna().mean() 

id                           0.000000
candidate                    0.006921
candidate_confidence         0.000000
relevant_yn                  0.000000
relevant_yn_confidence       0.000000
sentiment                    0.000000
sentiment_confidence         0.000000
subject_matter               0.023502
subject_matter_confidence    0.000000
candidate_gold               0.997981
name                         0.000000
relevant_yn_gold             0.997693
retweet_count                0.000000
sentiment_gold               0.998919
subject_matter_gold          0.998702
text                         0.000000
tweet_coord                  0.998486
tweet_created                0.000000
tweet_id                     0.000000
tweet_location               0.282027
user_timezone                0.317425
dtype: float64

In [22]:
data.drop(['candidate_gold','relevant_yn_gold','sentiment_gold','subject_matter_gold','tweet_coord'],axis=1,inplace=True)

In [79]:
df = data[['text','sentiment']]
df = df[df.sentiment != "Neutral"]
df

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [80]:
df['text'].apply((lambda x: re.sub('[a-zA-z0-9\s]','',x)))

1        @:'#.'.#://./…
3            @:-----#'…
4          @:#/@.#://.…
5             @:@:"."#@
6            @:..#@://…
              ...      
13866       @:❤️❤️❤️❤️#
13867        @:######@…
13868           @:#,,.#
13869          @:#'.""…
13870      @:#.!!#://./
Name: text, Length: 10729, dtype: object

In [81]:
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
df['text'] = df['text'].apply(lambda x: x.replace('rt',''))
df

,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbott_tx tedcruz on my first day i will ...,Positive
6,warriorwoman91 i liked her and was happy when...,Negative
...,...,...
13866,cappy_yarbrough love to see men who will neve...,Negative
13867,georgehenryw who thought huckabee exceeded th...,Positive
13868,lrihendry tedcruz as president i will always ...,Positive
13869,jrehling gopdebate donald trump says that he ...,Negative


In [82]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [106]:
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)
X

array([[   0,    0,    0, ..., 1308, 1394,  733],
       [   0,    0,    0, ...,  231,  713,   17],
       [   0,    0,    0, ...,  204,  366,  678],
       ...,
       [   0,    0,    0, ...,   71,   65,    3],
       [   0,    0,    0, ..., 1007, 1406,   74],
       [   0,    0,    0, ...,  194,    3,  712]])

In [107]:
Counter(tokenizer.word_index).most_common(5)

[('laura4faiax', 15766),
 ('exceeded', 15765),
 ('georgehenryw', 15764),
 ('notme', 15763),
 ('distance', 15762)]

In [108]:
X.shape, len(tokenizer.word_index)

((10729, 28), 15766)

In [111]:
from keras.layers import Embedding, LSTM, SpatialDropout1D, Dense
from keras import Sequential

In [114]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(2000, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 28, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [115]:
from sklearn.model_selection  import train_test_split

In [117]:
Y = pd.get_dummies(df['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [118]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
225/225 - 73s - loss: 0.4375 - accuracy: 0.8151
Epoch 2/7
225/225 - 49s - loss: 0.3209 - accuracy: 0.8667
Epoch 3/7
225/225 - 47s - loss: 0.2772 - accuracy: 0.8880
Epoch 4/7
225/225 - 47s - loss: 0.2472 - accuracy: 0.8972
Epoch 5/7
225/225 - 47s - loss: 0.2219 - accuracy: 0.9065
Epoch 6/7
225/225 - 48s - loss: 0.1947 - accuracy: 0.9207
Epoch 7/7
225/225 - 51s - loss: 0.1790 - accuracy: 0.9256


In [119]:
model.save(r'../Model/model.h5')

In [120]:
def txt_seq(txt):
    txt = tokenizer.texts_to_sequences(txt)
    txt = pad_sequences(txt)
    return txt

In [140]:
Bad = txt_seq('Meetings: Because none of us is as dumb as all of us.')
Good = txt_seq('scottwalker didnt catch the full gopdebate last night here are some of scotts best lines in 90 seconds walker16 httptcozsff')

In [141]:
np.argmax(model.predict(Bad))

0

In [142]:
np.argmax(model.predict(Good))

220